**Author:**
Jean Paul Azzopardi

**Dates:**
March - November 2023

# **Instructions:**

The cells below contain code that connects to the MassCourt MySQL database.

The tables should be visible as is. If not, navigate to 'Runtime' above and press 'Run all.'

The notebook will execute accordingly - **it will take some time (~5 to 10 minutes)**. After the intial run, the data will be cached so it won't take as long to load again.

Once done, you will be able to see interactive tables (if not available click the magic wand next to the table and it will make it interactive). Please note the tables are limited to 200,000 rows, even though queries may intend to return more. You can filter and order as you would in an Excel file.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.2 MB/s eta 0:00:00


In [ ]:
import pymysql
import pandas as pd
import numpy as np
from google.colab import data_table

In [ ]:
data_table.enable_dataframe_formatter()

In [ ]:
#Connect to MySQL database

cnx = pymysql.connect(user='buspark', password='U@5p1r3!',
                              host='masscourtsplus.org',
                              database='wp_courtdocs') #REMEMBER LOGIN


cursor = cnx.cursor()
# more info here: https://docs.google.com/document/d/1aZKF0X7HjcXUYurD8uQYDpgo9wQdeJH4CMviBJeTYVc/edit#

In [ ]:

query =  """
DESCRIBE cdocs_case_action_index;|

"""

# **TABLE 9: CASELOAD PER COURT, Grouped By Year**

In [ ]:
# How many cases have been filed each year over the last ten years in each court?
query =  """
 SELECT
    SUBSTRING(file_date, 1, 4) AS case_year,
    wp_terms.name AS court,
    COUNT(*) AS count
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    wp_terms ON wp_terms.term_id = cdocs_meta.court_term_id -- joining court
        JOIN
    wp_term_taxonomy ON wp_term_taxonomy.term_id = cdocs_meta.court_term_id
        JOIN
    wp_terms t1 ON t1.term_id = wp_term_taxonomy.parent -- joining parent court
        JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = cdocs_meta.post_id
WHERE
    t1.name LIKE '%District%'
    AND SUBSTRING(file_date, 1, 4) >= '2014'
GROUP BY
    SUBSTRING(file_date, 1, 4), wp_terms.name, t1.name
ORDER BY
    case_year ASC, count DESC;
      """

%time
cursor.execute(query) # executing query

results = cursor.fetchall() # fetching result

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.06 µs


OperationalError: (2013, 'Lost connection to MySQL server during query')

In [ ]:
case_per_court_by_year = pd.DataFrame(results)
case_per_court_by_year.rename({0:'Year', 1:'Court',2:'Count'}, axis = 1)

# **TABLE 9: Organization And Methods**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def organize(x):
  #Obtains the maximum/minimum year and an array filled with all years in between
  min_y = int(x[1][0])
  max_y = int(x[-1][0])
  years_arr = np.linspace(min_y, max_y, (max_y - min_y + 1),dtype=int)
  court_count = {}
  for i in range(1,len(x)):
    #iterates through the table
    court = x[i][1]
    index = abs((min_y - int(x[i][0])))
    if court not in court_count:
      #Creates a new dictionary reference with an arr len == years_arr
      court_count[court] = ([-1] * len(years_arr))
      court_count[court][index] = x[i][-1]
    else:
      #updates the existing dictionaries
      court_count[court][index] = x[i][-1]
  return years_arr, court_count

def line_graph(years_arr, court_count):
    # Plotting the data for each court
    plt.figure(figsize=(20, 12))
    for court, counts in court_count.items():
      if(sum(counts)/len(counts)>2000):
        plt.plot(years_arr, counts, marker='o', linestyle='-', label=court)

    # Adding titles and labels
    plt.title('Number of Cases Over The Years for Different Courts')
    plt.xlabel('Year')
    plt.ylabel('Number of Cases')

    # Adding legend
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # Adding grid
    plt.grid(True)

    # Show plot
    plt.savefig("Number_of_case_per_year.png")
    plt.show()

In [ ]:
years_arr, court_count = organize(results)
line_graph(years_arr, court_count)

Note:
Data is incomplete for 2024

In [ ]:
import matplotlib.pyplot as plt

class Stat:
    def __init__(self, years_arr, court_count):
        self.years_arr = years_arr
        self.court_count = court_count
        self.limiter = 0

    def total_cases(self):
        total = sum(sum(counts) for counts in self.court_count.values())
        return total

    def most_cases_per_court_per_year(self):
        most = {}
        for y in range(len(self.years_arr)):
            year = self.years_arr[y]
            m = 0
            year_i = abs(self.years_arr[0] - year)
            court = ""

            for key in self.court_count:
                if self.court_count[key][year_i] > m:
                    court = key
                    m = self.court_count[key][year_i]
            most[year] = [court, m]
        return most

    def print_most_cases_per_court_per_year(self):
        # Call the method most_cases_per_court_per_year and prints the results
        x = self.most_cases_per_court_per_year()
        for key in x:
            print("The most cases in", str(key), "is", str(x[key][0]), "with", x[key][-1], "cases")
            print()

    def average_amount_cases_per_court(self):
        average = {}
        for key in self.court_count:
            # Iterates through all the courts
            avg = 0
            for v in range(len(self.court_count[key])):
                avg += self.court_count[key][v]
            avg = int(avg/len(self.court_count[key]))
            average[key] = avg
        return average

    def print_average_cases_per_court_per_year(self):
        # Call the method most_cases_per_court_per_year and prints the results
        x = self.average_amount_cases_per_court()
        for key in x:
            print("The average amount of cases in the last 10 years for", str(key), "is", str(x[key]), "cases")
            print()

    def bar_graph_most(self, dic, title, x_axis, y_axis):
        # Inputs a dic, title's for graph, x_axis and y_axis
        x = list(dic.keys())
        y = list(dic.values())
        print(x)
        print(y)
        return 0

    def bar_graph_average(self, data, title, x_axis, y_axis):
        # Inputs a dic, title's for graph, x_axis and y_axis
        years = list(data.keys())
        x_axis =  list(data.keys())
        y =  list(data.values())
        return 0
        # Plotting
        plt.figure(figsize=(10, 6))
        plt.bar(x, y)
        plt.title(title)
        plt.xlabel(x_axis)
        plt.ylabel(y_axis)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()


    def bar_graph_most(self, dic, title, x_axis, y_axis):
        # Extracting data
        x = list(dic.keys())
        y = list(dic.values())
        label = [-1] * len(y)
        colors = {}

        # Extracting labels and values
        for i in range(len(y)):
            # Splitting up the tuples in dic.values
            label[i] = y[i][0]
            y[i] = y[i][-1]
            # Assigning unique color for each label
            if label[i] not in colors:
                colors[label[i]] = plt.cm.tab10(len(colors))
        # Plotting
        plt.figure(figsize=(10, 6))
        bars = plt.bar(x, y, color=[colors[l] for l in label])
        plt.title(title)
        plt.xlabel(x_axis)
        plt.ylabel(y_axis)
        plt.xticks(x, rotation=45, ha='right')  # Using years as x-axis labels
        plt.tight_layout()

        # Creating legend
        legend_handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in colors]
        plt.legend(legend_handles, colors.keys())

        # Show plot
        plt.show()


    def pie_chart_sum_total(self):
        w = self.court_count
        l = len(self.court_count)
        k = [-1] * l
        a = 0
        for key in w:
            k[a] = sum(w[key])
            a += 1
        labels = list(self.court_count.keys())
        plt.figure(figsize=(12, 12))
        plt.pie(k, labels=labels, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 10})  # Adjust label fontsize
        plt.axis('equal')
        plt.title('Pie Chart of Total Cases by Court')
        plt.legend(loc='best', fontsize='small', title='Courts', title_fontsize='small')  # Adjust legend fontsize
        # Adding legend
        plt.legend(loc='center left', bbox_to_anchor=(1.25, 0.5))

        # Adding grid
        plt.grid(True)
        plt.show()

    def total_case_per_year(self):
      #returns an array that's length is equal to the range of years with the toal sum of cases
      tyears = [0] * len(self.years_arr)
      for i in range(len(self.years_arr)):
        for k in self.court_count.keys():
          if (self.court_count[k][i] > self.limiter):
            if(tyears[i] == -1):
              tyears[i] = self.court_count[k][i]
            else:
              tyears[i] += self.court_count[k][i]
      return tyears

    def pie_chart_year(self, year,filename):
        minx = max(self.years_arr) - year
        k = [-1] * len(self.court_count)
        a = 0
        s = 0
        for i in self.court_count:
            k[a] = self.court_count[i][minx]
            a += 1
            s += self.court_count[i][minx]
        labels = list(self.court_count.keys())
        median = s / len(k)  # Calculate the median
        llabels = []
        percentages = []
        limiter = .015  # Adjust as needed
        for i in range(len(k)):
            if k[i] > median:
                llabels.append(labels[i])
                percentages.append(k[i]/s)
        plt.figure(figsize=(8, 8))
        plt.pie(percentages, labels=llabels, autopct='%1.1f%%')
        plt.title(f'Number of Court Cases by District above the median number of cases in {year}')
        try:
          plt.savefig(filename)
          plt.close()
          print(f'Pie chart saved as {filename}')
        except Exception as e:
          print(f'Error saving pie chart: {e}')



In [ ]:
# Execute
years_arr, court_count = organize(results)
# Creating an instance of the Stat class
statistics = Stat(years_arr, court_count)

print(statistics.total_case_per_year())

In [ ]:
#To Save plt
#statistics.pie_chart_year(2022,'pie_chart_year2018.png')

In [ ]:
# Getting total cases in the last 10 years
total_cases = statistics.total_cases()
print("There was a total of", str(total_cases), "cases in the last ten years")
print()

# Graphs
# Most cases per year
title = "The most amount of cases per year by court"
x_axis = "Year"
y_axis = "Number of Cases"



statistics.bar_graph_most(statistics.most_cases_per_court_per_year(),title,x_axis,y_axis)

# Average amount of cases
title2 = "Average amount of cases over the last ten years"
x_axis2 = "Court"
y_axis2 = "Average Number of Cases"


#Finish at a later date
#statistics.bar_graph_average(statistics.average_amount_cases_per_court(),title2,x_axis2,y_axis2)

# Sum total of cases
statistics.pie_chart_sum_total()



# Prints()
  # Average case number over the last ten y
# statistics.print_average_cases_per_court_per_year()

  # Most in the last ten years
# statistics.print_most_cases_per_court_per_year()


# **Table 10: Pro Se Cases by Courts**

In [ ]:

# caseload by attorneys, case type, and count for all cases
query = """
SELECT
    case_number,
    SUBSTRING(cdocs_meta.file_date, 1, 4) AS case_year,
    wp_posts.post_title AS attorney,
    wp_terms.name AS court
FROM
    cdocs_case_meta_index AS cdocs_meta
    JOIN wp_terms ON wp_terms.term_id = cdocs_meta.court_term_id
    JOIN wp_term_taxonomy ON wp_term_taxonomy.term_id = cdocs_meta.court_term_id
    JOIN wp_terms AS t1 ON t1.term_id = wp_term_taxonomy.parent
    JOIN cdocs_party_assignment_index ON cdocs_party_assignment_index.case_id = cdocs_meta.post_id
    JOIN wp_posts ON wp_posts.id = cdocs_party_assignment_index.attorney_id
WHERE
    wp_posts.post_type = 'Attorney'
    AND wp_posts.post_title LIKE '%Pro %'
    AND SUBSTRING(cdocs_meta.file_date, 1, 4) >= '2014'
GROUP BY
    wp_posts.post_title, wp_terms.name, SUBSTRING(cdocs_meta.file_date, 1, 4), case_number
ORDER BY
    SUBSTRING(cdocs_meta.file_date, 1, 4) ASC;
"""

#Attorney Id is not made by the stuff

%time

cursor.execute(query)  # executing query

results2 = cursor.fetchall()  # fetching result

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [ ]:
table_10= pd.DataFrame (results2)
table_10 = table_10.rename({0:'Case Number', 1:'Year', 2:'Attorney',3:'Court',4:'Count'}, axis = 1)
table_10['Case Number'].nunique()

285963

In [ ]:
import pandas as pd

unique_default_cases = table_10['Case Number'].unique()
# Filter the DataFrame to include only rows with unique case numbers
unique_cases_deftault = table_10[table_10['Case Number'].isin(unique_default_cases)]
unique_cases_deftault

In [ ]:

unique_case_numbers_prose = unique_cases_prose['Case Number'].nunique()
print("Total Pro Se Debt Collection Cases:", unique_case_numbers_prose)

# **Table 10 Organization And Methods**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def Table_10_organzize(x):
  #Takes In Table 10 and creates a dictionary based on location,
  #in which the length of the array stored is the years

  #Obtains the maximum/minimum year and an array filled with all years in between
  min_y = int(x[1][0])
  max_y = int(x[-1][0])
  years_arr = np.linspace(min_y, max_y, (max_y - min_y + 1),dtype=int)
  court_count = {}
  for i in range(1,len(x)):
    #iterates through the table
    court = x[i][2]
    index = abs((min_y - int(x[i][0])))
    if court not in court_count:
      #Creates a new dictionary reference with an arr len == years_arr
      court_count[court] = ([-1] * len(years_arr))
      court_count[court][index] = x[i][-1]
    else:
      #updates the existing dictionaries
      court_count[court][index] = x[i][-1]
  return years_arr, court_count




def Table10_comparison(Table10,Other,Limit):
  #Takes in two tables and creates a bar graph to represent difference over time
  #Limit is ant integer that will not add anything below to the comparison chart
  #Returns a bar graph in which Table 10 is orange and other table is blue
  years_arr, court_count = Table_10_organzize(results2)
  Oyears, Ocourt = organize(results)
  if(Limit <= 0):
    #Make sure the limit is set to zero so it doesn't include
    Limit = 0
  plt.figure(figsize=(20, 12))
  return 0



def Table10_total_comparison(years,court,Limit):
  #Takes in two tables and creates a bar graph to represent difference over time
  #Limit is ant integer that will not add anything below to the comparison chart
  #Returns a bar graph in which Table 10 is orange and other table is blue
  tyears = [-1] * len(years)
  for i in range(len(years)):
        for k in court.keys():
          if (court[k][i] > Limit):
            if(tyears[i] == -1):
              tyears[i] = court[k][i]
            else:
              tyears[i] += court[k][i]
  return tyears

def Table10_total_comparison_graph(x, other, years, filename,title = 'Total Cases vs Pro Se Cases', x_axis='Year', y_axis='Cases'):
    # Check if the lengths of x, other, and years are the same
    if len(x) != len(other) or len(x) != len(years):
        print("Length mismatch between arrays")
        return

    # Define colors for x and other bars
    colors = {'x': 'orange', 'other': 'blue'}

    plt.figure(figsize=(10, 6))

    # Define the width of each bar
    bar_width = 0.35

    # Calculate the position of the bars for x and other
    x_positions = np.arange(len(years))

    # Plot bars for x
    plt.bar(x_positions, x, width=bar_width, color=colors['x'], label='Pro Se')

    # Plot bars for other, starting from the right side of x bars
    plt.bar(x_positions + bar_width, other, width=bar_width, color=colors['other'], label='Total Cases')

    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.xticks(x_positions + bar_width / 2, years)  # Adjust x-axis ticks
    plt.legend()  # Show legend

    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

In [ ]:
#Execution
years_arr2, court_count2 = Table_10_organzize(results2)

#print(Table10_total_comparison(years_arr2,court_count2,0))

x = Table10_total_comparison(years_arr2,court_count2,0)
y = statistics.total_case_per_year()


print(sum(x))
print(x)
print(max(x))


Table10_total_comparison_graph(x,y,years_arr2,'Totalcases_vs_Pro_Se.png')


# **TABLE 11 Debt Cases in the last 10 years**


In [ ]:
# How many cases have been filed each year over the last ten years in each court?
query =  """
 SELECT
    SUBSTRING(file_date, 1, 4) AS case_year,
    wp_terms.name AS court,
    COUNT(*) AS count
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    wp_terms ON wp_terms.term_id = cdocs_meta.court_term_id -- joining court
        JOIN
    wp_term_taxonomy ON wp_term_taxonomy.term_id = cdocs_meta.court_term_id
        JOIN
    wp_terms t1 ON t1.term_id = wp_term_taxonomy.parent -- joining parent court
        JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = cdocs_meta.post_id
WHERE
    t1.name LIKE '%District%'
    AND SUBSTRING(file_date, 1, 4) >= '2014'
    AND action_idx.description LIKE '%debt%'
GROUP BY
    SUBSTRING(file_date, 1, 4), wp_terms.name, t1.name
ORDER BY
    case_year ASC, count DESC;
LIMIT 100
      """

%time
cursor.execute(query) # executing query

results = cursor.fetchall() # fetching result

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


InterfaceError: (0, '')

In [ ]:
debt_cases = pd.DataFrame(results)
debt_cases.rename({0:'Year', 1:'Court',2:'Count'}, axis = 1)

Cross Referencing with case_claims.csv

In [ ]:
import pandas as pd

case_claims = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS506/Project/data/case_claims.csv',usecols= ['case_number'])
print(case_claims)

In [ ]:
query =  """
 SELECT
    cdocs_meta.case_number,
    SUBSTRING(file_date, 1, 4) AS case_year,
    wp_terms.name AS court,
    COUNT(*) AS count
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    wp_terms ON wp_terms.term_id = cdocs_meta.court_term_id -- joining court
        JOIN
    wp_term_taxonomy ON wp_term_taxonomy.term_id = cdocs_meta.court_term_id
        JOIN
    wp_terms t1 ON t1.term_id = wp_term_taxonomy.parent -- joining parent court
        JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = cdocs_meta.post_id
WHERE
    t1.name LIKE '%District%'
    AND SUBSTRING(file_date, 1, 4) >= '2014'
      """

%time
cursor.execute(query) # executing query

results3 = cursor.fetchall() # fetching result

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [ ]:
table_11= pd.DataFrame (results3)
table_11 = table_10.rename({0:'Case Number', 1:'Year', 2:'Court',4:'Count'}, axis = 1)
print(table_11)

          Case Number  Year         Attorney             Court
0         BE14P0454EA  2014  Pro Se (PROPER)  Berkshire County
1         BE14P0359EA  2014  Pro Se (PROPER)  Berkshire County
2       14H79CV000333  2014  Pro Se (PROPER)           Western
3       14H79CV000335  2014  Pro Se (PROPER)           Western
4       14H79CV000336  2014  Pro Se (PROPER)           Western
...               ...   ...              ...               ...
285958    BR24D0498JP  2024           Pro Se    Bristol County
285959    BR24D0499JP  2024           Pro Se    Bristol County
285960    BR24P0949EA  2024           Pro Se    Bristol County
285961  24H79SP001674  2024           Pro Se           Western
285962   2460CV000125  2024           Pro Se           Wareham

[285963 rows x 4 columns]


#**Close Connection MYSQL server**

In [ ]:
cursor.close() # closing connection to MySQL server

cnx.close() # closing connection to MySQL server